## Análise da base de dados `Beta` utilizando algoritmos de ML

Neste notebook será analisado o `Beta dataset` utilizando algoritmos de ML para realizar a (1) extração de características, (2) seleção de características e (3) classificação dos dados

### Pontos importantes do dataset

- Frequências estimuladas (total de 40, com a diferença de 0.2 Hz uma da outra): 8.0, 8.2, ..., 15.6, 15.8;
- Taxa de amostragem: 250 Hz

### Analisar os "momentos" em que ocorrem evocação do sinal SSVEP

1. Criar o objeto `MNE` a partir dos dados dados do participante;
2. Aplicar no objeto `MNE` o filtro passa-faixa nos valores de 6 - 18 Hz;
3. Criar cópias do objeto `MNE` com fatias de tempo menores para analisar momentos que ocorrem estimulos ou não (verificar artigo);
    a) 0.0 - 0.5 segundos e 2.5 - 3.0 segundos ocorre apenas ruído;
    b) 0.5 - 2.5 segundos ocorre sinal SSVEP (com ruídos)
4. Com os sinais separados em objetos `MNE`, aplicar a `FFT`, para que seja possível plotar gráficos que contenham (ou não) as informações.
    - Os dados devem ser plotados no domínio da frequência (após a transformada de Fourier). O FFT pode ser realizado pela biblioteca `scipy.fft`.
    - Deve ser observado que as janelas (a) com ruído não aparecerão de fato o sinal SSVEP.

### Extração de características

Uma característica importante de acordo com o artigo base do dataset `BETA` é o *signal-to-noise ratio* (SNR).
São dois tipos de características SNR que podem ser implementadas: SNR de banda estreita (`narrow-SNR`) e SNR de banda larga (`wide-band SNR`).

Uma boa prática, é considerar o ruído das medidas de `SNR`, uma vez que os dados `SSVEP` não estão estimulados durante os períodos de 0 a 0,5 segundos e de 2,5 a 3 segundos. O ruído pode afetar a precisão das medidas de `SNR` e, portanto, é aconselhável levar isso em consideração.

Vamos realizar todos esses cálculos com dados fictícios:

In [32]:
# Criando dados fictícios

import numpy as np

shape = (160, 64, 750)
data = np.random.normal(loc=0, scale=10, size=shape).astype(np.float32)
data = np.load("./datasets/beta/data1.npy")
data.shape

(160, 64, 750)

Agora iremos estimar o ruído de fundo, para calcular posteriormente o `narrow SNR` e o `wide-band SNR`. 

In [33]:
# Estimando o ruído de fundo

# intervalos de tempo sem estímulo (0 a 0,5 segundos e 2,5 a 3 segundos)
base_start = 0
base_end = 125
rest_start = 625
rest_end = 750

# armazena uma lista com as médias de potência para cada canal
noise_power = []
# consideramos a primeira amostra (1º target, 1º trial)
for channel_data in data[0, :, :]:
    fft_result = np.fft.fft(channel_data)
    # densidade espectral de potência (PSD)
    psd = np.abs(fft_result) ** 2
    # média da potência nos intervalos de tempo sem estímulo
    base_power = np.mean(psd[base_start:base_end])
    rest_power = np.mean(psd[rest_start:rest_end])
    # média das duas médias de potência obtidas anteriormente
    mean_noise_power = (base_power + rest_power) / 2
    noise_power.append(mean_noise_power)
#média das médias de potência de todos os canais para estimar o ruído de fundo
estimated_background_noise = np.mean(noise_power)
estimated_background_noise

15103018.633842528

Antes de calcular os SNRs, precisamos obter as amplitudes alvo por meio dos dados EEG:

In [34]:
from scipy.signal import find_peaks

sr = 250

# frequências alvo
target_frequencies = np.arange(8, 16, 0.2)
# lista para armazenar as amplitudes nas frequências alvo
target_amplitudes = []

for channel_data in data[0, :, :]:
    fft_result = np.fft.fft(channel_data)
    psd = np.abs(fft_result) ** 2
    frequencies = np.fft.fftfreq(len(fft_result), 1 / sr)
    target_amplitudes_trial = []
    for target_frequency in target_frequencies:
        # encontrando o índice da frequência alvo no espectro de frequência
        index = np.argmin(np.abs(frequencies - target_frequency))
        # amplitude na frequência alvo
        amplitude = np.sqrt(psd[index])
        target_amplitudes_trial.append(amplitude)
    target_amplitudes.append(target_amplitudes_trial)
target_amplitudes = np.array(target_amplitudes)
target_amplitudes.shape

(64, 40)

Vamos calcular o SNR de "banda estreita". Pode ser observado pela seguinte equação:

$SNR_{banda\ estreita} = 10 \cdot \log_{10}\left(\frac{\text{energia total do espectro}}{\text{média das amplitudes nas frequências vizinhas}}\right)$

Já o SNR de banda larga é definido da seguinte forma:

$SNR_{banda\ larga} = 10 \cdot \log_{10}\left(\frac{\text{energia total do espectro}}{\text{energia total do espectro de amplitude}}\right)$

In [35]:
# forçando (estragando) valor de "estimated_background_noise" para não sobrar valores negativos
estimated_background_noise = 1.
target_amplitudes_adjusted = target_amplitudes - estimated_background_noise

# subtraindo o ruído de fundo das amplitudes
narrow_band_SNR = 10 * np.log10(target_amplitudes_adjusted / estimated_background_noise)
print(narrow_band_SNR)
print(narrow_band_SNR.shape)

total_power = np.sum(target_amplitudes_adjusted)
wide_band_SNR = 10 * np.log10(target_amplitudes_adjusted / total_power)
print(wide_band_SNR)
print(wide_band_SNR.shape)



[[25.77740934 26.74338683 26.74338683 ... 22.61651968 22.9069607
  22.9069607 ]
 [25.74636953 26.59396961 26.59396961 ... 22.71773438 22.99825634
  22.99825634]
 [25.64555961 26.40725916 26.40725916 ... 22.58648041 22.54358672
  22.54358672]
 ...
 [26.13694219 28.45707942 28.45707942 ... 22.44000692 24.17024884
  24.17024884]
 [26.65967043 27.32739704 27.32739704 ... 21.32528201 25.65420993
  25.65420993]
 [26.93082834 27.15088746 27.15088746 ... 21.43550063 25.77827242
  25.77827242]]
(64, 40)
[[-36.61548801 -35.64951052 -35.64951052 ... -39.77637767 -39.48593665
  -39.48593665]
 [-36.64652782 -35.79892774 -35.79892774 ... -39.67516297 -39.39464101
  -39.39464101]
 [-36.74733774 -35.98563819 -35.98563819 ... -39.80641694 -39.84931063
  -39.84931063]
 ...
 [-36.25595516 -33.93581793 -33.93581793 ... -39.95289043 -38.22264851
  -38.22264851]
 [-35.73322692 -35.0655003  -35.0655003  ... -41.06761534 -36.73868742
  -36.73868742]
 [-35.46206901 -35.24200989 -35.24200989 ... -40.95739672 -3

Ao final desta etapa, será obtido um vetor de características. Estas podem ser:
- `narrow SNR` (brigatória);
- `wide-band SNR` (brigatória);
- Maior valor espectral (FFT);
- Média dos valores espectrais (FFT).

Dimensionalidade dos dados será explicada da seguinte forma:

`40, 4, 64, 750` -> 40 targets, 4 trials, 64 canais e 750 valores
`160, 64 (SNR) + 64 (média) + 64 (maior) ...`
Resultando em `160, 192`.

### Seleção de características e classificação

Como existem diversos eletrodos (canais) que não obtém sinal SSVEP, podemos extrair as caracteríscas que não contribuem para a classificação dos dados.

Podemos utilizar o método `RFE` (*Recursive Feature Elimination*) aplicado por meio de `sklearn.feature_selection.RFE`, aprimorando o parâmetro `n_features_to_select` até obter o melhor resultado de classificação.

Para a classificação propriamente dita, é considerado o uso do método `SVM`.

# Código

In [50]:
import mne
import numpy as np
import matplotlib.pyplot as plt

data = np.load("datasets/beta/data1.npy")
data.shape
# (160, 64, 750)

(160, 64, 750)

In [51]:
n_channels = 64
sfreq = 250
ch_names = ["FP1", "FPZ", "FP2", "AF3", "AF4", "F7", "F5", "F3", "F1", "FZ", "F2", "F4", "F6", "F8", "FT7", "FC5", "FC3", "FC1", "FCZ", "FC2", "FC4", "FC6", "FT8", "T7", "C5", "C3", "C1", "CZ", "C2", "C4", "C6", "T8", "M1", "TP7", "CP5", "CP3", "CP1", "CPZ", "CP2", "CP4", "CP6", "TP8", "M2", "P7", "P5", "P3", "P1", "PZ", "P2", "P4", "P6", "P8", "PO7", "PO5", "PO3", "POZ", "PO4", "PO6", "PO8", "CB1", "O1", "OZ", "O2", "CB2"]
ch_types = ['eeg'] * n_channels
info = mne.create_info(ch_names, sfreq=sfreq, ch_types=ch_types)
from sklearn.preprocessing import LabelEncoder


labels = np.load("datasets/beta/labels1.npy")
labels, labels.shape


(array([ 8.6,  8.8,  9. ,  9.2,  9.4,  9.6,  9.8, 10. , 10.2, 10.4, 10.6,
        10.8, 11. , 11.2, 11.4, 11.6, 11.8, 12. , 12.2, 12.4, 12.6, 12.8,
        13. , 13.2, 13.4, 13.6, 13.8, 14. , 14.2, 14.4, 14.6, 14.8, 15. ,
        15.2, 15.4, 15.6, 15.8,  8. ,  8.2,  8.4,  8.6,  8.8,  9. ,  9.2,
         9.4,  9.6,  9.8, 10. , 10.2, 10.4, 10.6, 10.8, 11. , 11.2, 11.4,
        11.6, 11.8, 12. , 12.2, 12.4, 12.6, 12.8, 13. , 13.2, 13.4, 13.6,
        13.8, 14. , 14.2, 14.4, 14.6, 14.8, 15. , 15.2, 15.4, 15.6, 15.8,
         8. ,  8.2,  8.4,  8.6,  8.8,  9. ,  9.2,  9.4,  9.6,  9.8, 10. ,
        10.2, 10.4, 10.6, 10.8, 11. , 11.2, 11.4, 11.6, 11.8, 12. , 12.2,
        12.4, 12.6, 12.8, 13. , 13.2, 13.4, 13.6, 13.8, 14. , 14.2, 14.4,
        14.6, 14.8, 15. , 15.2, 15.4, 15.6, 15.8,  8. ,  8.2,  8.4,  8.6,
         8.8,  9. ,  9.2,  9.4,  9.6,  9.8, 10. , 10.2, 10.4, 10.6, 10.8,
        11. , 11.2, 11.4, 11.6, 11.8, 12. , 12.2, 12.4, 12.6, 12.8, 13. ,
        13.2, 13.4, 13.6, 13.8, 14. , 

In [57]:
# método para transformar labels categóricos
dataCateg = data
le = LabelEncoder()
events = np.column_stack((
    np.array(range(len(labels))),
    np.zeros(160, dtype=int),
    le.fit_transform(labels))
)

eventDict = {str(value): index  for index, value in enumerate(sorted(set(labels)))}
mne_data = mne.EpochsArray(dataCateg, info, events, event_id=eventDict)
filtered_mne_data = mne_data.filter(6, 18)
# filtered_mne_data

Not setting metadata
160 matching events found
No baseline correction applied
0 projection items activated
Setting up band-pass filter from 6 - 18 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 6.00
- Lower transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 5.00 Hz)
- Upper passband edge: 18.00 Hz
- Upper transition bandwidth: 4.50 Hz (-6 dB cutoff frequency: 20.25 Hz)
- Filter length: 413 samples (1.652 s)



[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done  71 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done 161 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done 287 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done 449 tasks      | elapsed:    0.1s
[Parallel(n_jobs=1)]: Done 647 tasks      | elapsed:    0.1s
[Parallel(n_jobs=1)]: Done 881 tasks      | elapsed:    0.1s
[Parallel(n_jobs=1)]: Done 1151 tasks      | elapsed:    0.2s
[Parallel(n_jobs=1)]: Done 1457 tasks      | elapsed:    0.2s
[Parallel(n_jobs=1)]: Done 1799 tasks      | elapsed:    0.3s
[Parallel(n_jobs=1)]: Done 2177 tasks      | elapsed:    0.3s
[Parallel(n_jobs=1)]: Done 2591 tasks      | elapsed:    0.4s
[Parallel(n_jobs=1)]: Done 3041 tasks      | elapsed:    0.4s
[Parallel(n_jobs=1)]: Done 3527 tasks      | elapsed:    0.5s
[Parallel(n_jobs=1)]: Done 4049 tasks      | elapsed:    0.5s
[Parallel(n_jobs=1)]: Done 4607 tasks      | elapsed:    0.6s
[Parallel(n_job

In [53]:
from scipy.signal import find_peaks

sr = 250

# Frequências alvo
target_frequencies = np.arange(8, 16, 0.2)
target_amplitudes = []

for channel_data in mne_data.get_data():
    target = []
    for eletrodo in channel_data:
        fft_result = np.fft.fft(eletrodo)
        psd = np.abs(fft_result) ** 2
        frequencies = np.fft.fftfreq(len(fft_result), 1 / sr)
        target_amplitudes_trial = []
        for target_frequency in target_frequencies:
            # Encontrando o índice da frequência alvo no espectro de frequência
            index = np.argmin(np.abs(frequencies - target_frequency))
            # Amplitude na frequência alvo
            amplitude = np.sqrt(psd[index])
            target_amplitudes_trial.append(amplitude)
        target.append(target_amplitudes_trial)
    
    target_amplitudes.append(target)
target_amplitudes = np.array(target_amplitudes)
target_amplitudes.shape

(160, 64, 40)

In [55]:
# Metodo do SNR
shape = data.shape

# intervalos de tempo sem estímulo (0 a 0,5 segundos e 2,5 a 3 segundos)
base_start = 0
base_end = 125
rest_start = 625
rest_end = 750

# armazena uma lista com as médias de potência para cada canal
noise_power = []
# consideramos a primeira amostra (1º target, 1º trial)
for channel_data in data[0, :, :]:
    fft_result = np.fft.fft(channel_data)
    # densidade espectral de potência (PSD)
    psd = np.abs(fft_result) ** 2
    # média da potência nos intervalos de tempo sem estímulo
    base_power = np.mean(psd[base_start:base_end])
    rest_power = np.mean(psd[rest_start:rest_end])
    # média das duas médias de potência obtidas anteriormente
    mean_noise_power = (base_power + rest_power) / 2
    noise_power.append(mean_noise_power)
#média das médias de potência de todos os canais para estimar o ruído de fundo
estimated_background_noise = np.mean(noise_power)
estimated_background_noise

9809606.831322439

In [56]:
# forçando (estragando) valor de "estimated_background_noise" para não sobrar valores negativos
estimated_background_noise = 1.
# Adicionado o np.abs() para remover erro de runtime
target_amplitudes_adjusted = np.abs(target_amplitudes - estimated_background_noise)

# subtraindo o ruído de fundo das amplitudes
narrow_band_SNR = 10 * np.log10(target_amplitudes_adjusted / estimated_background_noise)
print(narrow_band_SNR.shape)

total_power = np.sum(target_amplitudes_adjusted)
wide_band_SNR = 10 * np.log10(target_amplitudes_adjusted / total_power)
print(wide_band_SNR.shape)

(160, 64, 40)
(160, 64, 40)


In [48]:
data = np.array([narrow_band_SNR, wide_band_SNR])
print(data.shape)
# data = data.swapaxes(0, 1)
# data = data.swapaxes(1, 2)
# data = data.reshape(data.shape[0], data.shape[1], data.shape[2] * data.shape[3])
# print(data.shape)

# Reshapando o data para ficar no padrao correto

(2, 160, 64, 40)


In [43]:
import numpy as np
import mne
from sklearn.svm import SVC
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score


ch_ideal = ["PZ", "PO3", "PO5", "PO4", "PO6", "POZ", "O1", "OZ", "O2"] # canais ideais

# Encontre os índices dos canais ideais
indices = [ch_names.index(ch) for ch in ch_ideal]

# Use os índices para selecionar os canais desejados na matriz data
data_filtered = data[:, indices, :]

selected_ch = data_filtered.reshape(data_filtered.shape[0], -1)


In [44]:
data_reshape = data.reshape(data.shape[0], -1)
# (160, 5120)